# Reset the Knowledge Graph

## Imports

### Import python packages

To start, load some useful python packages, including some great stuff from langchain.
The "shared" notebook also sets up global constants like the connection to Neo4j,
embedding model to use, and the LLM to use for chat.


In [ ]:
%run 'shared.ipynb'

## Set up Neo4j

### Prepare a GraphDatabase driver

You can use the Neo4j `GraphDatabase` interface to send queries to the graph database.

In [ ]:
# expect `gdb` to be defined in shared notebook
# gdb = GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

result = gdb.execute_query("RETURN 'Hello, World!' AS message")

result.records[0].get('message')

# Clean up the graph to remove any existing data and indexes

Use these queries to reset the current graph to a blank state,
with no indexes, contraints or data.

In [ ]:
for constraint in gdb.execute_query('SHOW CONSTRAINTS').records:
    print(f"Removing constraint {constraint['name']}:")
    gdb.execute_query(f"DROP CONSTRAINT {constraint['name']}")

In [ ]:
for index in gdb.execute_query('SHOW INDEXES').records:
    print(f"Removing index {index['name']}:")
    gdb.execute_query(f"""
        DROP INDEX `{index['name']}`
    """)

In [ ]:
# Remove all data by matching any node, then "detach deleting" it,
# which means removing the node and all its relationships.
gdb.execute_query("""
        MATCH (all)
        DETACH DELETE all
""")

## Basic Cypher Queries

### Script - helpful way to show schema

In [ ]:
def show_schema(gdb):
  nodes = gdb.execute_query("""CALL db.schema.nodeTypeProperties()""")
  print(nodes)
  relationships = gdb.execute_query("""CALL db.schema.relTypeProperties()""")
  print(relationships)

show_schema(gdb)